<center>
<h1 style="font-family: 'Times New Roman', Times, serif; font-size: 60px;">DAA</h1>
<h2 style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">Stacking XGBoost</h2>
</center>

<font face="Times New Roman">

<h3 style="font-family: 'Times New Roman'">Imports</h3>

In [7]:
# Import necessary libraries
import pandas as pd                                                                                 # For data manipulation and analysis
import matplotlib.pyplot as plt                                                                     # For data visualization
import seaborn as sns                                                                               # For enhanced data visualization
from sklearn.model_selection import train_test_split                                                # For splitting data
import xgboost as xgb                                                                               # For the XGBoost
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer    # For evaluating model performance
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV                # For cross-validation
import os                                                                                           # For interacting with the operating system
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

<h3 style="font-family: 'Times New Roman'">Load All Datasets</h3>

In [8]:
final = pd.read_csv("Ficheiros/Ficheiros_Models/Final.csv", na_filter = False)
final_teste = pd.read_csv("Ficheiros/Ficheiros_Models/Final_Teste.csv")

<h3 style="font-family: 'Times New Roman'">XGBoost</h3>

In [9]:
X = final.drop(columns=['injection'])
y = final['injection']

In [10]:
# Define base models
rf_model = RandomForestClassifier(random_state=2024, n_estimators=1000, criterion='entropy', max_depth=25, min_samples_split=15, min_samples_leaf=1, bootstrap=True)
gb_model = GradientBoostingClassifier(random_state=2025, n_estimators=200, learning_rate=0.01, max_depth=5, subsample=0.9, min_samples_split=15)
et_model = ExtraTreesClassifier(random_state=2026, n_estimators=400, max_depth=50, min_samples_split=7, criterion='entropy', min_samples_leaf=1)
xgb_model = XGBClassifier(random_state=2027, learning_rate=0.09, n_estimators=150, max_depth=5, objective='multi:softprob')

# Define the stacking classifier with the chosen models as base estimators
stacking_model = StackingClassifier(
    estimators=[('rf', rf_model), ('gb', gb_model), ('et', et_model)],
    final_estimator=XGBClassifier(random_state=2028, learning_rate=0.09, n_estimators=150, max_depth=5, objective='multi:softprob'),
    cv=KFold(n_splits=10, shuffle=True)
)

# Train the stacking model on the training data
stacking_model.fit(X, y)

# Make predictions on the test set
X_test = final_teste
y_pred = stacking_model.predict(X_test)

In [11]:
reverse_mapping = {0: 'None', 1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}

# Aplica o mapeamento inverso às previsões 'y_pred'
y_pred = [reverse_mapping[pred] for pred in y_pred]

In [12]:
# Salvar as previsões em 'prediction_results.csv'
with open('Outputs/Stacking_XGBoost.csv', 'w') as file:
    file.write("RowId,Result\n")
    for row_id, prediction in enumerate(y_pred, start=1):
        file.write(f"{row_id},{prediction}\n")

print("Previsões salvas em Stacking_XGBoost.csv")

Previsões salvas em Stacking_XGBoost.csv
